In [ ]:
from pathlib import Path
import os
from ipynb.fs.full.notebook_viewer import Viewer
from scipy.spatial import distance
import numpy as np
from tqdm import tqdm

In [ ]:
root = Path("/data/natsuki/training116/00030-v4-mirror-auto4-gamma100-batch64-noaug-resumecustom/011289")
assert root.is_dir()
filelist = list(root.glob("*"))
filelist.sort()

In [ ]:
# dirty hack 4 load i2v
cwdir = os.getcwd()
i2vdir = Path("/home/natsuki/illustration2vec")
os.chdir(i2vdir)
import i2v
# %% 4096の特徴量使う版
model_feat = i2v.make_i2v_with_chainer("illust2vec_ver200.caffemodel")
model_tag = i2v.make_i2v_with_chainer("illust2vec_tag_ver200.caffemodel", "tag_list.json")
os.chdir(cwdir)

In [ ]:
from IPython.display import Javascript, display
import ipywidgets as wi
from PIL import Image
from io import BytesIO
from base64 import b64encode, b64decode
from uuid import uuid4
import numpy as np

def ID_canvas(ID=None, width=300, height=100):
    if not ID:
        ID = str(uuid4())[:8]
    canvas = wi.HTML(f"""
    <canvas
        id={ID}
        width={width}
        height={height}
        style=background:white;>
    </canvas>
    """)
    return ID, canvas

def encode(image):
    buffer = BytesIO()
    image.save(buffer, format="png")
    text = b64encode(buffer.getvalue()).decode()
    return f"data:image/png;base64,{text}"

def decode(text):
    return Image.open(BytesIO(b64decode(text.split(",")[-1])))
spylusjs = Path('/home/natsuki/MaGAN/spylus/spylus.js')

In [ ]:
fileupload = wi.FileUpload( accept='', multiple=False )
fileupload

In [ ]:
bdata = next(iter(fileupload.value.values()))["content"]
img = Image.open(BytesIO(bdata))

In [ ]:
ID, canvas = ID_canvas(width=512, height=512)
save_input = wi.Text(layout=wi.Layout(width="10%"))
paste_button = wi.Button(description="paste")
app = wi.VBox((
    save_input,
    paste_button,
    canvas,
))
display(app,
Javascript(f"""
{spylusjs.read_text()}
new (mix(
    Save, Paste
))({{
    canvas: document.getElementById("{ID}"),
    save_input: elem('{ID}', 'input', 0),
    paste_button: elem('{ID}', 'button', 0),
}});
"""))

In [ ]:
img = decode(save_input.value)

In [ ]:
illust2vec.estimate_plausible_tags([img], threshold=0)

In [ ]:
tagnpy = np.load(root/".."/"tag.npy")
filelist.sort()

In [ ]:
d = distance.euclidean
#d = distance.cosine
metrix = list()
target = tagnpy[1000]
for vec in tqdm(tagnpy):
    metrix.append( d(target, vec) )

In [ ]:
tmp = list(zip(metrix, filelist))
tmp.sort()
filelist_sorted = list(zip(*tmp))[1]

In [ ]:
Viewer(
    filelist_sorted,
    N=23,
    key="nosort",
    root=root,
)